In [12]:
# https://github.com/facebookresearch/hydra/blob/main/examples/jupyter_notebooks/compose_configs_in_notebook.ipynb

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
import os
from hydra import initialize_config_dir, compose
from omegaconf import OmegaConf

import napari
import napari
from squidpy.im import segment, ImageContainer


# this file acts as a robust starting point for launching hydra runs and multiruns
# can be run from any place

import pyrootutils

from napari_sparrow import utils

log = utils.get_pylogger(__name__)

# project root setup
# searches for root indicators in parent dirs, like ".git", "pyproject.toml", etc.
# sets PROJECT_ROOT environment variable (used in `configs/paths/default.yaml`)
# loads environment variables from ".env" if exists
# adds root dir to the PYTHONPATH (so this file can be run from any place)
# https://github.com/ashleve/pyrootutils
root = pyrootutils.setup_root(os.getcwd(), dotenv=True, pythonpath=True)

In [14]:
abs_config_dir = os.path.abspath(root / "configs")
with initialize_config_dir(version_base=None, config_dir=abs_config_dir):
    cfg = compose(config_name="segment", overrides=["+dataset=resolve"])
print(OmegaConf.to_yaml(cfg))

subset: []
task_name: segment
tags:
- dev
dataset:
  data_dir: ${paths.data_dir}/resolve/
  dtype: tiff
  image: ${dataset.data_dir}/20272_slide1_A1-1_DAPI.tiff
  obs: ${dataset.data_dir}/20272_slide1_A1-1_results.tiff
segmentation:
  method: watershed
paths:
  root_dir: ${oc.env:PROJECT_ROOT}
  data_dir: ${paths.root_dir}/data/
  log_dir: ${paths.root_dir}/logs/
  output_dir: ${hydra:runtime.output_dir}
  work_dir: ${hydra:runtime.cwd}
extras:
  ignore_warnings: false
  enforce_tags: true
  print_config: true



In [15]:
ic = ImageContainer(img=cfg.dataset.image)
ic

ImageContainer[shape=(12864, 10720), layers=['image']]

In [16]:
segment(ic, layer="image", method="watershed", lazy=True)
ic

ImageContainer[shape=(12864, 10720), layers=['image', 'segmented_watershed']]

In [17]:
seg = ic["segmented_watershed"].squeeze(dim=["z", "channels"]).data[:100, :100]
seg

dask.array<getitem, shape=(100, 100), dtype=uint32, chunksize=(100, 100), chunktype=numpy.ndarray>

In [18]:
img = ic["image"].squeeze(dim=["z", "channels"]).data[:100, :100]
img

dask.array<getitem, shape=(100, 100), dtype=uint16, chunksize=(100, 100), chunktype=numpy.ndarray>

In [19]:
viewer = napari.view_image(img)
# segmentation is slow
# viewer.add_labels(seg.compute())
napari.run()